In [1]:
%pylab inline
%gui qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
from spiketag.utils import EventEmitter

In [3]:
class state_monitor(EventEmitter):
    def __init__(self, _id):
        super(state_monitor, self).__init__()
        self._id = _id
        self.s = ['IDLE', 'BUSY', 'READY', 'DONE']
        self._state = self.s[0]
        
        
    @property
    def state(self):
        return self._state

    @state.setter
    def state(self, state):
        if state in self.s:
            self._state = state
            self.emit('report', state=self._state)
        else:
            print('state has to be one of', self.s)
            
    
    def set_busy(self):
        self.state = self.s[1]
        
    def set_ready(self):
        self.state = self.s[2]
        
    def set_done(self):
        self.state = self.s[3]

In [4]:
class status_manager(EventEmitter):
    def __init__(self):
        super(status_manager, self).__init__()
        self.reporters = {}
    
    def append(self, state_reporter):
        self.reporters[state_reporter._id] = state_reporter
        
        @state_reporter.connect
        def on_report(state):
#             print(state+' from group '+ str(state_reporter._id))
              self.emit('update', state=self.state_list)
            
    def __getitem__(self, i):
        return self.reporters[i]
    
    def __setitem__(self, i, state_str):
        self.reporters[i].state = state_str
    
    def __repr__(self):
        _str = ''
        for _, _reporter in self.reporters.items():
            _str = _str + str(_reporter._id) + ":" + str(_reporter.state) + '\n'
        return _str
    
    def reset(self):
        for reports in self.reporters.values():
            reports._state = 'IDLE'
        self.emit('update', state=self.state_list)
    
    @property
    def state_list(self):
        return [troy.s.index(troy.state) for troy in self.reporters.values()]

In [5]:
from spiketag.view import cluster_view

In [6]:
cluview = cluster_view()

In [7]:
s = status_manager()

In [8]:
troy = []
for i in range(10):
    troy.append(state_monitor(i))
for i in range(10):
    s.append(state_reporter = troy[i])

In [9]:
@s.connect
def on_update(state):
    print(state)
    cluview.set_data(10, nclu_list=10*[5], 
                     sorting_status=np.array(state), selected_group_id=1)

In [10]:
# disconnect if `update` function has something wrong
# s._callbacks.pop('update')

In [11]:
cluview.show()

In [ ]:
cluview.set_data(s)

In [12]:
id(troy[2]) == id(s[2])

True

In [28]:
troy[2].state = 'BUSY'
s[8] = 'READY'

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 2, 0]


In [25]:
s.reset()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
s.state_list

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
cluview.set_data(10, nclu_list=10*[5], 
                 sorting_status=np.array([1]*10), selected_group_id=1)